----
<img src="../Example.DataLibrary.Python-main/files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## RDP Middle Office News Use Case
Request company PermID, OrgName and statusPass for a large list of CUSIP, ISIN or SEDOLs:

Symbol conversion via RDP Symbology - **RDP Symbology permissions are required**

1. Read SEDOLs from file and chunk the list of SEDOLs
2. Map SEDOLs to PermIds
3. Request OrgName and Status


### Imports and Setup

In [26]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../Example.DataLibrary.Python-main/Configuration"

In [27]:
import refinitiv.data as rd
from refinitiv.data.delivery import endpoint_request
import asyncio
import pandas as pd
import numpy as np

In [28]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

### Read file with IDs
For tesing, we create a short list with defaultIDs and read in a long list of myIDs

In [29]:
defaultIDs =  ['BDZDRC5','BFZP189','BGY7Q00']
def readIDFile(filename):
    if filename != '':
        f = open(filename, "r")
        idList = f.read().splitlines()
        print('Read '+str(len(idList))+' IDs from file '+filename)
    else:
        idList = defaultIDss
    return idList
myIDs = readIDFile('SEDOLs.csv')

Read 2754 IDs from file SEDOLs.csv


### Chunk Long List into myIDChunks - Define a Helper Function 

In [30]:
CHUNK_SIZE = 500
def list_to_chunks(long_list, chunk_size):
    chunked_list = list()
    for i in range(0, len(long_list), chunk_size):
        chunked_list.append(long_list[i:i+chunk_size])
    return chunked_list
myIDChunks = list_to_chunks(myIDs, CHUNK_SIZE)
#print('My chunks are ',myIDChunks)
print('Number of chunks is: ',len(myIDChunks))

Number of chunks is:  6


### Open an RDP Session

In [42]:
# The session selected is required to have permissions 
# trapi.data.get.data.read, trapi.data.symbology.read
# contact your Refinitiv account manager to verify permissions in place
rd.open_session("desktop.workspace")
#rd.open_session(name='platform.rdp',config_name='../Example.DataLibrary.Python-main/Configuration/refinitiv-data.config.json')

<refinitiv.data.session.Definition object at 0x258048252e0 {name='workspace'}>

### Define Request Body Skeleton

In [43]:
# Example request with Body Parameter - Symbology Lookup
reqBodyPERMID  =  {         # Specify body parameters
        "from": [
            {
                "identifierTypes": [
                    "SEDOL"
                ],
                "values": [
                    
                ]
            }
        ],
        "to": [
            {
                "objectTypes": ["anyinstrument"],
                 "identifierTypes": ["PermID"]
            }
        ],
        "type": "auto"
    } 


### Request Lookup with defaultIDs
Enrich request body skeleton with defaultIDs

In [44]:
defaultIDs =  ['BDZDRC5','BFZP189','BGY7Q00']

reqBodyPERMID['from'][0]['values'] = defaultIDs 
request_definition = rd.delivery.endpoint_request.Definition(
    method = rd.delivery.endpoint_request.RequestMethod.POST,
    url = 'https://api.refinitiv.com/discovery/symbology/v1/lookup',
    body_parameters = reqBodyPERMID
)

response = request_definition.get_data()
response.data.raw
df = pd.json_normalize(response.data.raw,  record_path =['data'])
df

,input,output
0,"[{'value': 'BGY7Q00', 'identifierType': 'SEDOL'}]","[{'value': '21585425786', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]"
1,"[{'value': 'BDZDRC5', 'identifierType': 'SEDOL'}]","[{'value': '21528734875', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]"
2,"[{'value': 'BFZP189', 'identifierType': 'SEDOL'}]","[{'value': '21660454628', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]"


### Requests Lookup for Longer List of IDs
In chunks and display the single result 

In [45]:
dataAll = []
dfCompleteResult = pd.DataFrame()

for i in range(len(myIDChunks)):
    reqBodyPERMID['from'][0]['values'] = myIDChunks[i] 
    request_definition = rd.delivery.endpoint_request.Definition(
        method = rd.delivery.endpoint_request.RequestMethod.POST,
        url = 'https://api.refinitiv.com/discovery/symbology/v1/lookup',
        body_parameters = reqBodyPERMID
    )

    response = request_definition.get_data()
    response.data.raw
    df = pd.json_normalize(response.data.raw,  record_path =['data'])
    
    print("Requested chunk #", i)
#    print(df)
    dataAll.append(df)
dfCompleteResult = pd.concat(dataAll,ignore_index=True)
print('Displaying result:')
display(dfCompleteResult)
     

Requested chunk # 0
Requested chunk # 1
Requested chunk # 2
Requested chunk # 3
Requested chunk # 4
Requested chunk # 5
Displaying result:


,input,output,errors
0,"[{'value': 'B1CLYW7', 'identifierType': 'SEDOL'}]","[{'value': '8590929771', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN
1,"[{'value': 'BYXYF34', 'identifierType': 'SEDOL'}]","[{'value': '8590933028', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN
2,"[{'value': 'B16HTQ0', 'identifierType': 'SEDOL'}]","[{'value': '8589999123', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN
3,"[{'value': '2391092', 'identifierType': 'SEDOL'}]","[{'value': '8590923508', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN
4,"[{'value': 'BH3T5K7', 'identifierType': 'SEDOL'}]","[{'value': '8590937427', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN
5,"[{'value': 'B23PS12', 'identifierType': 'SEDOL'}]","[{'value': '8590016136', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN
6,"[{'value': '5730324', 'identifierType': 'SEDOL'}]","[{'value': '8590932594', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN
7,"[{'value': 'B063NY7', 'identifierType': 'SEDOL'}]","[{'value': '8590936959', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN
8,"[{'value': '2783815', 'identifierType': 'SEDOL'}]","[{'value': '8590937417', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN
9,"[{'value': 'B1CDZ19', 'identifierType': 'SEDOL'}]","[{'value': '8590922042', 'identifierType': 'PermID', 'objectType': 'EDInstrument'}]",NaN


### Extract PermIds and Request Org Names and Status

In [46]:
serAll = dfCompleteResult['output'].str.get(0)
permids = []
serAll
for val in serAll:
    if not pd.isna(val):
        permids.append(val.get('value'))
print(permids)


['8590929771', '8590933028', '8589999123', '8590923508', '8590937427', '8590016136', '8590932594', '8590936959', '8590937417', '8590922042', '8590927771', '8590929567', '8590920986', '21568435680', '21660454628', '21655477179', '46641209213', '8590937866', '8590927621', '8590940970', '8590932013', '21714622057', '8590006632', '8590934322', '25727309955', '8590935027', '8590931864', '8590929138', '8590940422', '8590926849', '8590926257', '8590059136', '21577420062', '8590920825', '8590938568', '8590938516', '21612525419', '21529526222', '21503758337', '8590925675', '8590926861', '8590931671', '8590936801', '21693338510', '8589970110', '192880974844', '21615062550', '8590922763', '8590923171', '8590941202', '8590930028', '21651361483', '8590935535', '8590925175', '8590923699', '8590925038', '8590938374', '8590928778', '8590935043', '8590943370', '8590920874', '8590938914', '8590928692', '21632919931', '8590855063', '21530246903', '25727426360', '8591400076', '8590936428', '8590922619', '

###  Request CommonName and OrganizationStatusCode - small list of Permids

In [47]:
# Requires permissions trapi.data.get.data.read, if absent will result in error 403 - access denied
# contact your Refinitiv account manager to verify permissions in place
rd.get_data(
    universe=['8590922809', '8590939219', '21581971624', '21483864933'],
    fields=['TR.CommonName','TR.OrganizationStatusCode','TR.Revenue']    
)

,Instrument,Company Common Name,Organization Status Code,Revenue
0,8590922809,Forward Air Corp,Act,1662427000
1,8590939219,Judges Scientific PLC,Act,91289000
2,21581971624,US Foods Holding Corp,Act,29487000000
3,21483864933,Carlyle Group Inc,Act,8782100000


###  Request CommonName and OrganizationStatusCode - Loop Permid Chunks
The request is partitioned into chunks of size N (default is 1000), as the full list of a larger size may result in an error

In [48]:
N = 1000
myPermidChunks = list_to_chunks(permids, N)
dataAll = []
dfAll = pd.DataFrame()
for i in range(len(myPermidChunks)):
    print('Iteration #',i)
    dfResp = rd.get_data(
        universe=myPermidChunks[i],
        fields=['TR.CommonName','TR.OrganizationStatusCode','TR.Revenue']    
    )
    dataAll.append(dfResp)
    #print(resp)
dfAll = pd.concat(dataAll,ignore_index=True)
print('Displaying full result:')
display(dfAll)

Iteration # 0
Iteration # 1
Iteration # 2
Displaying full result:


,Instrument,Company Common Name,Organization Status Code,Revenue
0,8590929771,National Health Investors Inc,Act,298715000
1,8590933028,Frontline Ltd,Act,749381000
2,8589999123,KWS SAAT SE & Co KgaA,Act,1310232000
3,8590923508,Winmark Corp,Act,78216200
4,8590937427,Enerpac Tool Group Corp,Act,528660000
5,8590016136,CVR Energy Inc,Act,7242000000
6,8590932594,Azkoyen SA,Act,138866000
7,8590936959,Tenaris SA,Act,6521207000
8,8590937417,Zimmer Biomet Holdings Inc,Act,7836200000
9,8590922042,Sekisui Chemical Co Ltd,Act,1056560000000


## Close the session

In [49]:
rd.close_session()